# 1. OpenAI의 챗봇 API 기초
## 입출력길이 제한 및 요금
토큰

In [ ]:
!pip install tiktoken==0.7.0

In [ ]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

### Tokenizer와 tiktoken

In [ ]:
import tiktoken

text = "LLM으로 멋진 것을 만드는 것은 쉽지만, 프로덕션에서 사용할 수 있는 것을 만드는 것은 매우 어렵다."

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

## (4) Chat Completions API를 테스트할 환경 준비하기
###OpenAI의 API 키 준비

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## (5) Chat Completions API 실습
### OpenAI의 라이브러리

In [ ]:
!pip install openai==1.54.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.40.6
    Uninstalling openai-1.40.6:
      Successfully uninstalled openai-1.40.6


### Chat Completions API 호출

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
	model="gpt-4o-mini",
	messages=[
		{"role": "system", "content": "You are a helpful assistant."},
		{"role": "user", "content": "안녕하세요! 저는 홍길동이라고 합니다."},
	],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-ATUtRyvIC7vStua7f1b540fo4TNEA",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731593629,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 27,
    "total_tokens": 48,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


### 대화 기록에 기반한 응답을 얻기


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
	model="gpt-4o-mini",
	messages=[
		{"role": "system", "content": "You are a helpful assistant."},
		{"role": "user", "content": "안녕하세요! 저는 홍길동이라고 합니다."},
		{"role": "assistant", "content": "안녕하세요, 홍길동님!. 만나서 반갑습니다. 어떻게 도와드릴까요?"},
		{"role": "user", "content": "제 이름을 아시나요?"},
	],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-ATV05JXGVjvd3OTqvbjuX3zPnfMAg",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "네, 홍길동님이라고 말씀해 주셨어요. 혹시 다른 질문이나 이야기가 있으신가요?",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731594041,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 63,
    "total_tokens": 89,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


### 스트리밍으로 응답을 얻기

In [ ]:
response = client.chat.completions.create(
	model="gpt-4o-mini",
	messages=[
		{"role": "system", "content": "You are a helpful assistant."},
		{"role": "user", "content": "안녕하세요! 저는 홍길동이라고 합니다."},
	],
	stream=True,
)

for chunk in response:
	content = chunk.choices[0].delta.content
	if content is not None:
		print(content, end="", flush=True)

안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?

### JSON 모드

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
	model="gpt-4o-mini",
	messages=[
		{
			"role": "system",
			"content": '인물목록을 다음 JSON형식으로 출력해주세요.\n{"people": ["aaa", "bbb"]}',
		},
		{
			"role": "user",
			"content": "옛날 옛적에 할아버지와 할머니가 있었다.",
		},
	],
	response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{"people": ["할아버지", "할머니"]}


### Vison (이미지 입력)

In [ ]:
from openai import OpenAI

client = OpenAI()

image_url = "https://llmdev.kr/langchain/panda.jpeg"

response = client.chat.completions.create(
	model="gpt-4o-mini",
	messages=[
		{
			"role": "user",
			"content": [
				{"type": "text", "text": "이미지를 설명해 주세요."},
				{"type": "image_url", "image_url": {"url": image_url}},
			],
		}
	],
)

print(response.choices[0].message.content)

이미지에는 귀여운 판다가 나와 있습니다. 판다는 검은색과 흰색의 털을 가진 동물로, 주로 대나무를 먹고 있습니다. 판다는 고개를 약간 돌려서 카메라를 바라보고 있으며, 주변에는 녹색 식물들이 있어 자연적인 환경이 느껴집니다. 판다의 표정은 귀여움이 가득하며, 전체적으로 아늑하고 평화로운 분위기를 자아냅니다.


## (6) Function calling
### Function calling 샘플 코드

In [ ]:
import json

def get_current_weather(location, unit="fahrenheit"):
	if "seoul" in location.lower():
		return json.dumps({"location": "Seoul", "temperature": "10", "unit": unit})
	elif "san francisco" in location.lower():
		return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
	elif "paris" in location.lower():
		return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
	else:
		return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
tools = [
	{
		"type": "function",
		"function": {
			"name": "get_current_weather",
			"description": "Get the current weather in a given location",
			"parameters": {
				"type": "object",
				"properties": {
					"location": {
						"type": "string",
						"description": "The city and state, e.g. San Francisco, CA",
					},
					"unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
				},
				"required": ["location"],
			},
		},
	}
]

In [ ]:
from openai import OpenAI

client = OpenAI()

messages = [
	{"role": "user", "content": "서울의 날씨는 어때요?"},
]

response = client.chat.completions.create(
	model="gpt-4o",
	messages=messages,
	tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-ATVs1rjlYB2QTv21zAIuVvB67dtqy",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_9Bll8lzTR9KjtnfnSkQISEbc",
            "function": {
              "arguments": "{\"location\":\"Seoul\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1731597385,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_45cf54deae",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 81,
    "total_tokens": 97,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
  

In [ ]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [ ]:
available_functions = {
	"get_current_weather": get_current_weather,
}

# 사용하고자 하는 함수는 여러개일 수 있으므로 루프처리
for tool_call in response_message.tool_calls:
	# 함수실행
	function_name = tool_call.function.name
	function_to_call = available_functions[function_name]
	function_args = json.loads(tool_call.function.arguments)
	function_response = function_to_call(
		location=function_args.get("location"),
		unit=function_args.get("unit"),
	)

	# 함수실행결과를 대화기록으로 messages에 추가함
	messages.append(
		{
			"tool_call_id": tool_call.id,
			"role": "tool",
			"name": function_name,
			"content": function_response,
		}
	)


In [ ]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "서울의 날씨는 어때요?"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_9Bll8lzTR9KjtnfnSkQISEbc",
        "function": {
          "arguments": "{\"location\":\"Seoul\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_9Bll8lzTR9KjtnfnSkQISEbc",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Seoul\", \"temperature\": \"10\", \"unit\": null}"
  },
  {
    "tool_call_id": "call_9Bll8lzTR9KjtnfnSkQISEbc",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Seoul\", \"temperature\": \"10\", \"unit\": null}"
  }
]


In [ ]:
from openai import OpenAI

client = OpenAI()

messages = [
	{"role": "user", "content": "서울의 날씨는 어때요?"},
]

second_response = client.chat.completions.create(
	model="gpt-4o",
	messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-ATWDynosYQmsx8EnVmsWO3NaveYcv",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "죄송하지만, 실시간 날씨 정보는 확인할 수 없습니다. 최신 서울 날씨는 기상청 웹사이트나 날씨 앱을 통해 확인하실 수 있습니다.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731598746,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_159d8341cc",
  "usage": {
    "completion_tokens": 37,
    "prompt_tokens": 16,
    "total_tokens": 53,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
